In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [2]:
df = pd.read_csv("C:/Users/shin jaeik/Documents/sas_data_project/DATA/TRAIN_DATA.csv",encoding= 'euc-kr')

In [5]:
# 결측치 행 제거(3개)
df = df.dropna()

# 문자열 컬럼 인코딩(라벨)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["AREA_NM_ENC"] = label_encoder.fit_transform(df['AREA_NM'])
df["DIST_NM_ENC"] = label_encoder.fit_transform(df['DIST_NM'])

# 시계열 데이터 변환
df['DATA_YM'] = pd.to_datetime(df['DATA_YM'], format='%Y%m')

# 연도, 월 분리
df['YEAR'] = df['DATA_YM'].dt.year
df['MONTH'] = df['DATA_YM'].dt.month

df['YM_NUM'] = df['YEAR'] * 12 + df['MONTH']

# X, y 나누기(타겟데이터 나누고, 문자열 컬럼 없애기)
X = df.drop(columns= ['TOTAL_ELEC', 'DATA_YM', 'AREA_NM', 'DIST_NM'])
y = df['TOTAL_ELEC']


# X셋도 시계열 데이터 변환
X['DATA_YM'] = pd.to_datetime(df['DATA_YM'], format='%Y%m')

# 연도, 월 분리
X['YEAR'] = X['DATA_YM'].dt.year
X['MONTH'] = X['DATA_YM'].dt.month

X['YM_NUM'] = X['YEAR'] * 12 + X['MONTH']

In [6]:
# 로그변환
df['LOG_ELEC'] = np.log1p(df['TOTAL_ELEC'])

X = df.drop(columns=['TOTAL_ELEC', 'DATA_YM', 'AREA_NM', 'DIST_NM'])
y = df['LOG_ELEC']

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size= 0.2 , random_state= 42)

# 객체 생성
model = LGBMRegressor(random_state=42)
# 모델 학습
model.fit(X_train, y_train)

y_pred_log = model.predict(X_val)

# 로그복원
y_pred = np.expm1(y_pred_log)
y_val_origin = np.expm1(y_val)

# RMSE 계산
rmse = mean_squared_error(y_val_origin, y_pred, squared= False)
print(f"로그복원 후 RMSE : {rmse:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012031 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2678
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 22
[LightGBM] [Info] Start training from score 6.500556
로그복원 후 RMSE : 142.2639
